In [2]:
import regex as re
import pandas as pd

In [3]:
f=open('./spark_15k.log')
lines=f.readlines()

In [21]:
split_pt=re.compile(r'([,\s$@<>\[\]\'"=]|\((?!\))|(?<!\()\)|[:\.](?=\s))')
hm_pt=re.compile(r'(^(?:\S+\s){4})(.*)')

In [22]:
header=[]
msg=[]
for line in lines:
    h,m=hm_pt.match(line).groups()
    
    header.append(h)
    msg.append(m)

In [23]:
print(header[0])
print(msg[0])

17/06/08 18:54:52 INFO python.PythonRunner: 
Times: total = 38, boot = -13, init = 51, finish = 0


In [24]:
header_tokens=[]
for h in header:
    tokens=[token for token in re.split(r'([\s$@<>\[\]\'"=]|\((?!\))|(?<!\()\)|[:\.,](?=\s)|(?<=\b[a-zA-Z]+):|:(?=[a-zA-Z]+\b))',h) if token!='' and token!=None]
    header_tokens.append(tokens)

In [25]:
msg_tokens=[]
for m in msg:
    tokens=[token for token in split_pt.split(m) if token!='' and token!=None]
    msg_tokens.append(tokens)

In [26]:
pd.DataFrame({'header_tokens':header_tokens,'msg_tokens':msg_tokens}).to_csv('tokens.csv')

In [27]:
header_anno=[]
for tokens in header_tokens:
    anno=['O']*len(tokens)
    anno[0],anno[2],anno[4],anno[6]='B-DATE','B-TIME','B-LVL','B-CLS'

    header_anno.append(anno)

In [28]:
list(zip(header_tokens[0],header_anno[0]))

[('17/06/08', 'B-DATE'),
 (' ', 'O'),
 ('18:54:52', 'B-TIME'),
 (' ', 'O'),
 ('INFO', 'B-LVL'),
 (' ', 'O'),
 ('python.PythonRunner', 'B-CLS'),
 (':', 'O'),
 (' ', 'O')]

In [29]:
msg_anno=[]
for tokens in msg_tokens:
    anno=['<O>']*len(tokens)
    msg_anno.append(anno)

In [30]:
re_date=r'\d{2}\/\d{2}\/\d{4}'
re_time=r'^\d{2}:\d{2}$'

re_obj=r'[a-zA-Z]+-\d{1,3}$'
re_cls=r'(\.([A-Z]+[a-z]*)+)|([A-Z]+[a-z]+){2,}$'
re_func=r'\S*\(\)$'
re_path=r'^\'?\/[a-zA-Z]'
re_ip=r'(?:[0-9]{1,3}\.){3}[0-9]{1,3}'

re_host=r':\d{3,5}$'
re_url=r'[a-z]+:\/\/\S*'


In [31]:
def sub_tag(ntokens,ntags, patt_tag_list):

    def _sub_tag(ntokens,ntags, patt, target_tag):
        for tokens,tags in zip(ntokens,ntags):
            for i in range(len(tokens)):
                if re.search(patt,tokens[i]):
                    tags[i]=target_tag

    for patt,tag in patt_tag_list:
        _sub_tag(ntokens,ntags,patt,tag)

In [32]:
sub_tag(msg_tokens,msg_anno,
    [(re_date,'B-DATE'),
    (re_time,'B-TIME'),
    (re_cls,'B-CLS'),
    (re_func,'B-FUNC'),
    (re_path,'B-PATH'),
    (re_ip,'B-IP'),
    (re_host,'B-HOST'),
    (re_url,'B-URL')])

In [33]:
list(zip(msg_tokens[0],msg_anno[0]))

[('Times', '<O>'),
 (':', '<O>'),
 (' ', '<O>'),
 ('total', '<O>'),
 (' ', '<O>'),
 ('=', '<O>'),
 (' ', '<O>'),
 ('38', '<O>'),
 (',', '<O>'),
 (' ', '<O>'),
 ('boot', '<O>'),
 (' ', '<O>'),
 ('=', '<O>'),
 (' ', '<O>'),
 ('-13', '<O>'),
 (',', '<O>'),
 (' ', '<O>'),
 ('init', '<O>'),
 (' ', '<O>'),
 ('=', '<O>'),
 (' ', '<O>'),
 ('51', '<O>'),
 (',', '<O>'),
 (' ', '<O>'),
 ('finish', '<O>'),
 (' ', '<O>'),
 ('=', '<O>'),
 (' ', '<O>'),
 ('0', '<O>')]

In [34]:
with open('./header_check.txt','w') as f:
    for tokens,tags in zip(header_tokens,header_anno):
        for token,tag in zip(tokens,tags):
            f.write(token+' '+tag+'\n')
        f.write('\n')

In [35]:
with open('./msg_check.txt','w') as f:
    for tokens,tags in zip(msg_tokens,msg_anno):
        for token,tag in zip(tokens,tags):
            f.write(token+' '+tag+'\n')
        f.write('\n')

In [36]:
df=pd.DataFrame({
    'tokens':[h+m for h,m in zip(header_tokens,msg_tokens)],
    'tags':[h+m for h,m in zip(header_anno,msg_anno)],
    'raw_log':[h+m for h,m in zip(header,msg)]
    })
df.to_csv('./spark_ds.csv')